# 빠른시작
[LINK](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html)

이번 장에서는 기계학습의 일반적인 작업들을 위한 API를 통해 실행됩니다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## 데이터 작업하기

파이토치(Pytorch)에는 *데이터 작업을 위한 기본 요소*두가지인 `torch.utils.DataLoader`와 `torch.utils.Dataset`이 있습니다. `Dataset`은 샘플과 정답(label)을 저장하고, `DataLoader`는 `Dataset`을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감쌉니다.

PyTorch는 *TorchText*, *TorchVision*, *TorchAudio*, *TorchScatter*와 같이 도메인 특화 라이브러리를 위한 몇가지 도구를 제공합니다. 이 튜토리얼에서는 *TorchVision*의 몇가지 기능을 사용합니다.

`torchvision.datasets`모듈은 CIFAR, COCO등과 같은 다양한 실제 비전(vision)데이터에 대한 `Dataset`을 포함하고 있습니다. 이 튜토리얼에서는 FasionMNIST 데이터셋을 사용합니다. 모든 TorchVision `Dataset`들은 샘플과 정답을 각각 변경하기 위한 `transform`과 `target_transform`을 인자로 받을 수 있습니다.

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

`Dataset`을 `DataLoader`의 인자로 전달합니다. 이는 `Dataset`을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle), 병렬 처리를 지원합니다. 여기서 배치 크기(batch size)는 튜토리얼의 편의를 위해 작은 값으로 설정합니다.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 모델 만들기

PyTorch에서 신경망 모델은 *nn.Module**을 상속받는 클래스(class)를 생성하여 정의합니다. `__init__`함수에서 신경망의 계층(layer)들을 정의하고 `forward`함수에서 신경망에 데이터를 어떻게 전달할지 지정합니다. 가능한 경우 GPU또는 MPS로 신경망을 이동시켜 연산을 가속(accelerate)할 수 있습니다.

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Using {} device".format(device))

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 모델 매개변수 최적화하기

모델을 학습하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요합니다.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 (배치(batch)로 제공되는)학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정합니다.

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch & 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인합니다.

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 단계는 여러번의 반복단계(epoch)로 이루어집니다. 각 epoch에서는 모델은 더 나은 예측을 하기 위해 매개변수를 학습합니다. 각 `epoch`마다 모델의 정확도(accuracy)와 손실(loss)을 출력합니다. `epoch`마다 정확도가 증가하고 손실이 감소하는 것을 보려고 합니다.

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.302672 [    0/60000]
loss: 2.304448 [   64/60000]
loss: 2.294259 [  128/60000]
loss: 2.295219 [  192/60000]
loss: 2.299822 [  512/60000]
loss: 2.296094 [  576/60000]
loss: 2.297063 [  640/60000]
loss: 2.295778 [  704/60000]
loss: 2.303056 [ 1024/60000]
loss: 2.306709 [ 1088/60000]
loss: 2.296635 [ 1152/60000]
loss: 2.298655 [ 1216/60000]
loss: 2.302291 [ 1536/60000]
loss: 2.304717 [ 1600/60000]
loss: 2.289470 [ 1664/60000]
loss: 2.295907 [ 1728/60000]
loss: 2.299078 [ 8192/60000]
loss: 2.288045 [ 8256/60000]
loss: 2.287491 [ 8320/60000]
loss: 2.301056 [ 8384/60000]
loss: 2.292219 [ 8704/60000]
loss: 2.291851 [ 8768/60000]
loss: 2.288779 [ 8832/60000]
loss: 2.283998 [ 8896/60000]
loss: 2.280465 [ 9216/60000]
loss: 2.288601 [ 9280/60000]
loss: 2.276414 [ 9344/60000]
loss: 2.296964 [ 9408/60000]
loss: 2.286409 [ 9728/60000]
loss: 2.289372 [ 9792/60000]
loss: 2.287019 [ 9856/60000]
loss: 2.297654 [ 9920/60000]
loss: 2.274833 [16384/60000]
los

## 모델 저장하기

모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함햐여) 내부 상태사전(internal state dictionary)을 직렬화(serialize)하는 것입니다.

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기
모델을 불러오는 과정에서 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됩니다.

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

모델을 사용하여 예측해봅니다.

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
